In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pandas as pd
import transformers
import torch
from transformers.utils import is_torch_available
import os
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
from transformers import AutoModel, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
datasets = {
    "ESOL": (df_esol, "label"),
    "FreeSolv"      : (df_freesolv,   "label"),
    "Lipophilicity" : (df_lipo,       "label"),
    "QM9 (alpha)"     : (df_qm9_alpha,  "label"),
    "QM9 (mu)"        : (df_qm9_mu,      "label"),
    "QM9 (A)"        : (df_qm9_A,      "label"),
    "QM9 (B)"        : (df_qm9_B,      "label"),
    "QM9 (C)"        : (df_qm9_C,      "label"),
    "QM9 (Homo)"        : (df_qm9_homo,      "label"),
    "QM9 (Lumo)"        : (df_qm9_lumo,      "label"),
    "QM9 (HL Gap)"        : (df_qm9_gap,      "label"),
    "QM9 (r2)"        : (df_qm9_r2,      "label"),
    "QM9 (zpve)"        : (df_qm9_zpve,      "label"),
    "QM9 (u0)"        : (df_qm9_u0,      "label"),
    "QM9 (u298)"        : (df_qm9_u298,      "label"),
    "QM9 (h298)"        : (df_qm9_h298,      "label"),
    "QM9 (g298)"        : (df_qm9_g298,      "label"),
    "QM9 (CV)"        : (df_qm9_cv,      "label"),
    "QM9 (u0 atom)"        : (df_qm9_u0_atom,  "label"),
    "QM9 (u298 atom)"        : (df_qm9_u298_atom,      "label"),
    "QM9 (h298 atom)"        : (df_qm9_h298_atom,      "label"),
    "QM9 (g298 atom)"        : (df_qm9_g298_atom,      "label"),
    "MP tryg." : (df_mptri, "label"),
    "MP Molecules": (df_mp, "label"),
    "Dyn. Visc.": (df_dynvisc, "label"),
    "Free E. cyclo": (df_cyclo, "label"),
    "Monomer Densities": (df_Densities, "label"),
    "Monomer Rgyr": (df_Rgyr, "label"),
    "Monomer Tg": (df_Tg, "label"),
    "Monomer Ecoh": (df_Ecoh, "label"),
}


In [ ]:
model_dirs = [
    "bert-base-uncased",
    "DeepChem/ChemBERTa-77M-MLM",
    'DeepChem/ChemBERTa-77M-MTR',
    'ibm-research/MoLFormer-XL-both-10pct',
    "allenai/scibert_scivocab_uncased",
    "allenai/scibert_scivocab_cased",
    "answerdotai/ModernBERT-base",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-1n4g-no-tags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-500k-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-1M5-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-10-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-20-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-30-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-40-epochs-notags",
    # "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-50-epochs-notags",
    "/home/david/modernbert_chemistry/fineweb/fine-web-modernbert-base-8192-multi-tok-new-procedure-60-epochs-notags",

]

model_titles = [
    "bert-base-uncased",
    "ChemBERTa-MLM",
    "ChemBERTa-MTR",
    "MoLFormer",
    "SciBERT_uncased",
    "SciBERT_cased",
    "ModernBERT (MB)",
    # "ModernBERT base 500k",
    # "ModernBERT base Procedures 500k",
    # "ModernBERT base Procedures 1M5 5 epochs",
    # "ModernBERT base Procedures 1M5 10 epochs",
    # "ModernBERT base Procedures 1M5 20 epochs",
    # "ModernBERT base Procedures 1M5 30 epochs",
    # "ModernBERT base Procedures 1M5 40 epochs",
    # "ModernBERT base Procedures 1M5 50 epochs",
    "ModernBERT base Procedures 1M5 60 epochs"
]


def get_cls_embeddings(tokenizer, model, texts, batch_size=32, max_length=512):
    """Extract CLS embeddings for a list of texts."""
    all_embeddings = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i : i + batch_size]
            inputs = tokenizer(batch, padding="longest", truncation=True,
                               return_tensors="pt", max_length=max_length)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs, output_hidden_states=True)
            cls_embeds = outputs.hidden_states[-1][:, 0, :].cpu().numpy()
            all_embeddings.append(cls_embeds)
    return np.vstack(all_embeddings)

n_experiments = 3
all_results_reg = {}

for model_dir, title in zip(model_dirs, model_titles):
    print(f"\nModel: {title} ")
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForMaskedLM.from_pretrained(model_dir, output_hidden_states=True)
    model.to(device)
    
    model_tasks = {}
    for task_name, (df_task, label_col) in datasets.items():
        print(f"Task: {task_name}")
        # Prepare data
        texts = df_task["SMILES"].astype(str).tolist()
        y_cont = df_task[label_col].astype(float).values
        
        X_embeds = get_cls_embeddings(tokenizer, model, texts)
        
        r2_scores, mse_scores = [], []
        for run in range(n_experiments):
            X_train, X_test, y_train, y_test = train_test_split(
                X_embeds, y_cont, test_size=0.2, random_state=42 + run
            )
            reg = RandomForestRegressor(
                n_estimators=100,
                random_state=42 + run,
                n_jobs=-1     # <- use all cores
            )
            # reg = XGBRegressor(
            #     n_estimators=100,
            #     random_state=42 + run,
            #     objective="reg:squarederror",  # standard regression objective
            #     verbosity=0
            # )
            reg.fit(X_train, y_train)
            preds = reg.predict(X_test)
            r2  = r2_score(y_test, preds)
            mse = mean_squared_error(y_test, preds)
            r2_scores.append(r2)
            mse_scores.append(mse)
        mean_r2  = np.mean(r2_scores)
        std_r2   = np.std(r2_scores)
        mean_mse = np.mean(mse_scores)
        std_mse  = np.std(mse_scores)
        
        print(f"  R²:  {mean_r2:.3f} ± {std_r2:.3f}")
        print(f"  MSE: {mean_mse:.3f} ± {std_mse:.3f}")
        
        model_tasks[task_name] = {
            "r2_scores": r2_scores,
            "mse_scores": mse_scores,
            "mean_r2": mean_r2,
            "std_r2": std_r2,
            "mean_mse": mean_mse,
            "std_mse": std_mse
        }
    
    all_results_reg[title] = model_tasks

rows = []
for model, tasks in all_results_reg.items():
    for task, metrics in tasks.items():
        rows.append({
            "model": model,
            "task": task,
            "mean_r2": metrics["mean_r2"],
            "std_r2": metrics["std_r2"],
            "mean_mse": metrics["mean_mse"],
            "std_mse": metrics["std_mse"]
        })

results_df_reg = pd.DataFrame(rows)

